In [207]:
import xarray as xr
import numpy as np
import glob
import os.path

from dask.distributed import Client
from dask.diagnostics import ProgressBar

from aggfly import dataset, regions, grid_weights
from aggfly.aggregate import TemporalAggregator, SpatialAggregator, get_time_dim

ProgressBar().register()
# client = Client()

In [208]:
# Set file output name/path
output_path = "/home3/dth2133/data/aggregated/counties/"
output_name = "usa_counties_corn_monthly"
output_varn = "sine"
csv = True

In [209]:
# Open shapefile containing region features.
georegions = regions.from_name('counties')

# Open example climate dataset to calculate grid weights.
clim = dataset.from_path(
    f"/home3/dth2133/data/usa/usa-t2m_tempPrecLand2019.zarr", 
    't2m', 
    'zarr', 
    preprocess=dataset.timefix_era5l)

# Clip climate data to the US (raw data are global)
clim.clip_data_to_georegions_extent(georegions)

# Rechunk dataset to optimize multithreading
clim.rechunk((5, 578, -1, -1, -1, -1))

# Calculate area and crop layer weights.
weights = grid_weights.from_objects(clim, georegions, crop='corn')

# This object covers aggregating hourly and daily data to the yearly 
# level
daily = [TemporalAggregator(
    'sine',
    agg_from='hour',
    agg_to='day', 
    ddargs=[x,999,0]) for x in [10]]
# daily = annual = TemporalAggregator(
#     'sum', 
#     agg_from='day',
#     agg_to='year')


annual = TemporalAggregator(
    'sum', 
    agg_from='day',
    agg_to='month')

# This object aggregates cells within a region to the average across 
# cells, weighted by `weights`, which in this case are the area of the
# cell and the share of the cell with corn crops.
spatial = SpatialAggregator('avg')


In [210]:
# list(range(-3,35))
clim.da

,Array,Chunk
Bytes,4.81 GiB,98.43 MiB
Shape,"(250, 578, 1, 12, 31, 24)","(5, 578, 1, 12, 31, 24)"
Count,3640 Tasks,50 Chunks
Type,float32,numpy.ndarray


In [5]:
# Calculate the grid weights
w = weights.weights()

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 22.6s
[########################################] | 100% Completed | 21.8s
[########################################] | 100% Completed | 42.4s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


In [211]:
def aggregate_era5l_t2m_multi(path):
    
    # path = "/home3/dth2133/data/usa/usa-t2m_tempPrecLand1969.zarr"
    # Open climate dataset.
    clim = dataset.from_path(
        path, 
        't2m', 
        'zarr', 
        preprocess=dataset.timefix_era5l) # Kelvin to Celsius
    
    # Clip climate data to the US (raw data are global)
    clim.clip_data_to_georegions_extent(georegions)
    # Rechunk dataset to optimize multithreading
    clim.rechunk((5, 578, -1, -1, -1))
    
    daily_list = [x.map_execute(clim.da.data) for x in daily]
    annual_list = [annual.map_execute(x) for x in daily_list]
    rc_list = [x.rechunk(-1) for x in annual_list]
    spatial_list = [spatial.map_execute(x, w) for x in rc_list]
    
    return spatial_list
    

In [212]:
# Check years from the input path - this just creates a vector of years for
# which my raw climate data are available, e.g. [1970, 1971, ...]
import numpy as np
import glob
from os.path import basename
files = np.sort([x for x in glob.glob('/home3/dth2133/data/usa/*t2m_*')])
# Loop over years and aggregate.
output = list()
for f in files:
    print(f)
    output.append(aggregate_era5l_t2m_multi(f))

/home3/dth2133/data/usa/usa-t2m_tempPrecLand1951.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1952.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1953.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1954.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1955.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1956.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1957.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1958.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1960.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1961.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1962.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1963.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1964.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1965.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1966.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1967.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1968.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1969.zarr
/home3/dth2133/data/usa/usa-

In [213]:
import dask
result = dask.compute(output)

[########################################] | 100% Completed | 37min  3.3s
[########################################] | 100% Completed | 37min  3.3s


In [214]:
lr = ['sine'+str(x).replace('-','M') for x in [10]]
lr

['sine10']

In [233]:
temp = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))

In [235]:
len(result[0][0])
dlist = list()
for d in range(0, len(result[0][0])):
    ylist = list()
    for y in range(0, len(result[0])):
        ylist.append(result[0][y][d])  
    temp_array = xr.DataArray(
        data = np.concatenate(ylist, axis=1),
        dims = ['region', 'year', 'month'],
        coords = dict(
            region=('region', georegions.regions),
            year = ('year', temp.year.values),
            month = ('month', temp.month.values))).to_dataset(name = lr[d])
    dlist.append(temp_array)

In [237]:
outds = xr.combine_by_coords(dlist)

In [239]:
ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
# ds = ds.drop(lr)
# ds.compute()
ds = xr.combine_by_coords([ds,outds]).compute()

[########################################] | 100% Completed |  4.8s
[########################################] | 100% Completed |  4.8s


In [241]:
# https://github.com/pydata/xarray/issues/3476
for v in list(ds.coords.keys()):
    if ds.coords[v].dtype == object:
        ds.coords[v] = ds.coords[v].astype("unicode")

for v in list(ds.variables.keys()):
    if ds[v].dtype == object:
        ds[v] = ds[v].astype("unicode")
        
ds.to_zarr(os.path.join(output_path, output_name+'.zarr'), mode='w')

In [242]:
ds

<xarray.Dataset>
Dimensions:  (region: 3108, year: 68, month: 12)
Coordinates:
  * region   (region) <U5 '01001' '01003' '01005' ... '56041' '56043' '56045'
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year     (year) int64 1951 1952 1953 1954 1955 ... 2015 2016 2017 2018 2019
Data variables: (12/50)
    dd08     (region, year, month) float64 97.64 128.0 216.3 ... 5.062 0.1649
    dd10     (region, year, month) float64 64.49 94.92 167.7 ... 2.201 0.01085
    dd29     (region, year, month) float64 0.0 0.0 0.0 0.0009577 ... 0.0 0.0 0.0
    dd30     (region, year, month) float64 0.0 0.0 0.0 0.0 ... 1.215 0.0 0.0 0.0
    dd36     (region, year, month) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    prec     (region, year, month) float64 1.122 1.45 1.851 ... 0.4905 0.1758
    ...       ...
    time8    (region, year, month) float64 18.33 17.48 25.25 ... 1.945 0.1252
    time9    (region, year, month) float64 16.62 16.52 24.27 ... 1.42 0.06682
    timeM1   (region, year, month) float64 30.02 24.82 31.0 ... 15.51 9.025
    timeM2   (region, year, month) float64 30.38 25.27 31.0 ... 18.21 10.96
    timeM3   (region, year, month) float64 30.86 25.95 31.0 ... 20.55 13.45
    sine10   (region, year, month) float64 77.53 106.6 181.5 ... 3.266 0.01853

In [243]:
if csv:
    ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
    ds.to_dataframe().to_csv(os.path.join(output_path, output_name+'.csv'))

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100